In [1]:
import pandas as pd
import chardet
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support, roc_auc_score, roc_curve, auc, f1_score
from sklearn.preprocessing import OneHotEncoder
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_selection import RFECV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
import numpy as np
import math
import joypy
from imblearn.under_sampling import RandomUnderSampler, EditedNearestNeighbours, TomekLinks
from imblearn.combine import SMOTETomek, SMOTEENN
from collections import Counter
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate 
from sklearn.feature_selection import SelectFromModel
from datetime import datetime
import matplotlib.ticker as ticker


# py -m pip install matplotlib

In [2]:
df_private_MTPL = pd.read_csv('df_private_mtpl.txt', sep="\t")

In [3]:
df_private_MTPL.head(5)

PRODUCT Duration  Vehicle_age  Make    GWP vehicle_type_group_mapped  \
0     MOD     Long          7.0    VW  38.67               6) Personal   
1     MOD     Long          4.0   KIA  24.59               6) Personal   
2     MOD     Long          4.0   KIA  24.55               6) Personal   
3     MOD     Long         11.0  FORD  31.42               6) Personal   
4     MOD     Long         11.0  FORD  31.44               6) Personal   

   number_of_claims  Churn  Renewal  Policy_Duration  Gender  Client_Age  \
0               0.0      0        1            364.0    Male        38.0   
1               1.0      0        1            364.0  Female        45.0   
2               1.0      0        1            364.0  Female        45.0   
3               1.0      0        1            364.0    Male        38.0   
4               1.0      0        1            364.0    Male        38.0   

      City  Years_of_driving  
0    Other              18.0  
1  Vilnius              24.0  
2  Vilnius              24.0  
3  Vilnius              18.0  
4  Vilnius              18.0

In [4]:
df_private_MTPL['Vehicle_age'] = pd.to_numeric(df_private_MTPL['Vehicle_age'], errors='coerce')
df_private_MTPL['GWP'] = pd.to_numeric(df_private_MTPL['GWP'], errors='coerce')
df_private_MTPL['number_of_claims'] = pd.to_numeric(df_private_MTPL['number_of_claims'], errors='coerce')
df_private_MTPL['Client_Age'] = pd.to_numeric(df_private_MTPL['Client_Age'], errors='coerce')
df_private_MTPL['Policy_Duration'] = pd.to_numeric(df_private_MTPL['Policy_Duration'], errors='coerce')
df_private_MTPL['Years_of_driving'] = pd.to_numeric(df_private_MTPL['Years_of_driving'], errors='coerce')

df_private_MTPL['PRODUCT'] = pd.Categorical(df_private_MTPL['PRODUCT'])
df_private_MTPL['Duration'] = pd.Categorical(df_private_MTPL['Duration'])
df_private_MTPL['Make'] = pd.Categorical(df_private_MTPL['Make'])
df_private_MTPL['vehicle_type_group_mapped'] = pd.Categorical(df_private_MTPL['vehicle_type_group_mapped'])
df_private_MTPL['City'] = pd.Categorical(df_private_MTPL['City'])
df_private_MTPL['Churn'] = pd.Categorical(df_private_MTPL['Churn'])
df_private_MTPL['Renewal'] = pd.Categorical(df_private_MTPL['Renewal'])
df_private_MTPL['Gender'] = pd.Categorical(df_private_MTPL['Gender'])


In [5]:
df_private_MTPL2 = df_private_MTPL

numerical_columns = ['Vehicle_age', 'GWP', 'number_of_claims', 'Policy_Duration', 'Client_Age', 'Years_of_driving']

scaler = MinMaxScaler()
df_private_MTPL2[numerical_columns] = scaler.fit_transform(df_private_MTPL2[numerical_columns])

In [6]:
df_private_MTPL3 = df_private_MTPL2

categorical_columns = ['PRODUCT', 'Duration', 'Make', 'vehicle_type_group_mapped', 'Gender', 'City']

encoder = OneHotEncoder(sparse_output=False, handle_unknown='error')

X_encoded = encoder.fit_transform(df_private_MTPL3[categorical_columns])

encoded_column_names = encoder.get_feature_names_out(categorical_columns)

df_encoded = pd.DataFrame(X_encoded, columns=encoded_column_names)

df_private_MTPL3 = pd.concat([df_private_MTPL3.drop(columns=categorical_columns), df_encoded], axis=1)

In [7]:
X = df_private_MTPL3.drop(columns=['Churn']) 
y = df_private_MTPL3['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

X_train_ht, X_test_ht, y_train_ht, y_test_ht = train_test_split(X_train, y_train, stratify=y_train, test_size=0.7, random_state=42)

In [10]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import ADASYN
from imblearn.combine import SMOTETomek
from sklearn.metrics import fbeta_score
from sklearn.metrics import make_scorer, roc_auc_score, f1_score, precision_recall_curve, precision_recall_fscore_support

In [11]:
f1_scorer_beta2 = make_scorer(fbeta_score,
                          beta=2,
                          greater_is_better = True,
                          pos_label=0)

In [12]:
# NESUBALANSUOTI DUOMENYS

param_dist = {
    'n_estimators': [100, 500, 1000],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [2, 4, 6, 8, 10],
    'criterion': ['gini', 'entropy', 'log_loss']
}

rf = RandomForestClassifier(random_state=42)

random_search = RandomizedSearchCV(rf, param_distributions=param_dist, n_iter=60, cv=5, scoring=f1_scorer_beta2, random_state=42)

random_search.fit(X_train_ht, y_train_ht)

print("Best Parameters found:")
print(random_search.best_params_)

KeyboardInterrupt: 

In [14]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

In [ ]:
# NESUBALANSUOTI DUOMENYS

param_dist = {
    'n_estimators': [100, 500, 1000],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [2, 4, 6, 8, 10],
    'criterion': ['gini', 'entropy', 'log_loss']
}

rf = RandomForestClassifier(random_state=42)

halving_search = HalvingRandomSearchCV(estimator=rf, param_distributions=param_dist, cv=5, scoring=f1_scorer_beta2, random_state=42)
#random_search = RandomizedSearchCV(rf, param_distributions=param_dist, n_iter=60, cv=5, scoring=f1_scorer_beta2, random_state=42)

random_search.fit(X_train_ht, y_train_ht)

print("Best Parameters found:")
print(random_search.best_params_)

# Under sampling metodai

- RandomUnderSampler()
- EditedNearestNeighbours()
- TomekLinks()

In [16]:
f1_scorer = make_scorer(f1_score)

In [17]:
rus =  RandomUnderSampler()
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

class_counts_resampled = pd.Series(y_train_rus).value_counts()

print("Class Counts after RandomUnderSampler:")
print(class_counts_resampled)


X_train_ht_rus, X_test_ht_rus, y_train_ht_rus, y_test_ht_rus = train_test_split(X_train_rus,y_train_rus, stratify=y_train_rus, test_size=0.7, random_state=42)



Class Counts after RandomUnderSampler:
Churn
0    76427
1    76427
Name: count, dtype: int64


In [18]:
param_dist = {
    'n_estimators': [100, 500, 1000],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [2, 4, 6, 8, 10],
    'criterion': ['gini', 'entropy', 'log_loss']
}

rf = RandomForestClassifier(random_state=42)

random_search = RandomizedSearchCV(rf, param_distributions=param_dist, cv=5, scoring=f1_scorer, random_state=42)

random_search.fit(X_train_ht_rus, y_train_ht_rus)

print("Best Parameters found:")
print(random_search.best_params_)

print("Best cross-validation accuracy: {:.3f}".format(random_search.best_score_))

Best Parameters found:
{'n_estimators': 500, 'min_samples_split': 6, 'min_samples_leaf': 4, 'max_features': None, 'max_depth': 60, 'criterion': 'entropy'}
Best cross-validation accuracy: 0.839


In [ ]:
############################################

In [20]:
enn =  EditedNearestNeighbours()
X_train_enn, y_train_enn = enn.fit_resample(X_train, y_train)

class_counts_resampled = pd.Series(y_train_enn).value_counts()

print("Class Counts after RandomUnderSampler:")
print(class_counts_resampled)


X_train_ht_enn, X_test_ht_enn, y_train_ht_enn, y_test_ht_enn = train_test_split(X_train_enn,y_train_enn, stratify=y_train_enn, test_size=0.7, random_state=42)



Class Counts after RandomUnderSampler:
Churn
0    823725
1     76427
Name: count, dtype: int64


In [21]:
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [2, 4, 6, 8, 10],
    'criterion': ['gini', 'entropy', 'log_loss']
}

rf = RandomForestClassifier(random_state=42)

random_search = RandomizedSearchCV(rf, param_distributions=param_dist, cv=5, scoring=f1_scorer, random_state=42)

random_search.fit(X_train_ht_enn, y_train_ht_enn)

print("Best Parameters found:")
print(random_search.best_params_)

print("Best cross-validation accuracy: {:.3f}".format(random_search.best_score_))

In [ ]:
##############################################################

# Over sampling metodai

- RandomOverSampler()
- SMOTE()
- ADASYN()

In [ ]:
ros =  RandomOverSampler()
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

class_counts_resampled = pd.Series(y_train_ros).value_counts()

print("Class Counts after RandomUnderSampler:")
print(class_counts_resampled)


X_train_ht_ros, X_test_ht_ros, y_train_ht_ros, y_test_ht_ros = train_test_split(X_train_ros,y_train_ros, stratify=y_train_ros, test_size=0.7, random_state=42)



In [ ]:
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [2, 4, 6, 8, 10],
    'criterion': ['gini', 'entropy', 'log_loss']
}

rf = RandomForestClassifier(random_state=42)

random_search = RandomizedSearchCV(rf, param_distributions=param_dist, cv=5, scoring=f1_scorer, random_state=42)

random_search.fit(X_train_ht_ros, y_train_ht_ros)

print("Best Parameters found:")
print(random_search.best_params_)

print("Best cross-validation accuracy: {:.3f}".format(random_search.best_score_))

In [ ]:
############################################################################

In [ ]:
sm =  SMOTE()
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)

class_counts_resampled = pd.Series(y_train_sm).value_counts()

print("Class Counts after RandomUnderSampler:")
print(class_counts_resampled)


X_train_ht_sm, X_test_ht_sm, y_train_ht_sm, y_test_ht_sm = train_test_split(X_train_sm,y_train_sm, stratify=y_train_sm, test_size=0.7, random_state=42)



In [ ]:
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [2, 4, 6, 8, 10],
    'criterion': ['gini', 'entropy', 'log_loss']
}

rf = RandomForestClassifier(random_state=42)

random_search = RandomizedSearchCV(rf, param_distributions=param_dist, cv=5, scoring=f1_scorer, random_state=42)

random_search.fit(X_train_ht_sm, y_train_ht_sm)

print("Best Parameters found:")
print(random_search.best_params_)

print("Best cross-validation accuracy: {:.3f}".format(random_search.best_score_))

In [ ]:
##########################################################################

# Combined sampling metodai

- SMOTEENN()
- SMOTETomek()


In [ ]:
smt =  SMOTETomek()
X_train_smt, y_train_smt = smt.fit_resample(X_train, y_train)

class_counts_resampled = pd.Series(y_train_smt).value_counts()

print("Class Counts after RandomUnderSampler:")
print(class_counts_resampled)


X_train_ht_smt, X_test_ht_smt, y_train_ht_smt, y_test_ht_smt = train_test_split(X_train_smt, y_train_smt, stratify=y_train_smt, test_size=0.7, random_state=42)



In [ ]:
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [2, 4, 6, 8, 10],
    'criterion': ['gini', 'entropy', 'log_loss']
}

rf = RandomForestClassifier(random_state=42)

random_search = RandomizedSearchCV(rf, param_distributions=param_dist, cv=5, scoring=f1_scorer, random_state=42)

random_search.fit(X_train_ht_smt, y_train_ht_smt)

print("Best Parameters found:")
print(random_search.best_params_)

print("Best cross-validation accuracy: {:.3f}".format(random_search.best_score_))